###### Bullet Graphs

In [13]:
from bokeh.io import show, output_notebook
from bokeh.palettes import PuBu4
from bokeh.plotting import figure
from bokeh.models import Label

output_notebook()

Loading BokehJS ...

In [14]:
data= [("John Smith", 105, 120),
       ("Jane Jones", 99, 110),
       ("Fred Flintstone", 109, 125),
       ("Barney Rubble", 135, 123),
       ("Mr T", 45, 105)]

limits = [0, 20, 60, 100, 160]
labels = ["Poor", "OK", "Good", "Excellent"]
cats = [x[0] for x in data]

In [15]:
cats

['John Smith', 'Jane Jones', 'Fred Flintstone', 'Barney Rubble', 'Mr T']

In [23]:
p=figure(title="Sales Rep Performance", plot_height=350, plot_width=800, y_range=cats)
p.x_range.range_padding = 0
p.grid.grid_line_color = None
p.xaxis[0].ticker.num_minor_ticks = 0

In [32]:
zip(limits[:-1], limits[1:], PuBu4[::-1])

In [34]:
print(tuple(zip(limits[:-1], limits[1:], PuBu4[::-1])))

((0, 20, '#f1eef6'), (20, 60, '#bdc9e1'), (60, 100, '#74a9cf'), (100, 160, '#0570b0'))


In [25]:
for left, right, color in zip(limits[:-1], limits[1:], PuBu4[::-1]):
    p.hbar(y=cats, left=left, right=right, height=0.8, color=color)

In [26]:
show(p)

In [35]:
perf = [x[1] for x in data]
p.hbar(y=cats, left=0, right=perf, height=0.3, color="black")

GlyphRenderer(id='1466', ...)

In [36]:
show(p)

In [37]:
comp = [x[2]for x in data]
p.segment(x0=comp, y0=[(x, -0.5) for x in cats], x1=comp,
          y1=[(x, 0.5) for x in cats], color="white", line_width=2)

GlyphRenderer(id='1603', ...)

In [38]:
show(p)

In [39]:
for start, label in zip(limits[:-1], labels):
    p.add_layout(Label(x=start, y=0, text=label, text_font_size="10pt",
                       text_color='black', y_offset=5, x_offset=15))

In [40]:
show(p)

###### Waterfall Chart

In [41]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models.formatters import NumeralTickFormatter
import pandas as pd

output_notebook()

Loading BokehJS ...

In [42]:
# Create the initial dataframe
index = ['sales','returns','credit fees','rebates','late charges','shipping']
data = {'amount': [350000,-30000,-7500,-25000,95000,-7000]}
df = pd.DataFrame(data=data,index=index)

# Determine the total net value by adding the start and all additional transactions
net = df['amount'].sum()

In [43]:
net

375500

In [44]:
df

,amount
sales,350000
returns,-30000
credit fees,-7500
rebates,-25000
late charges,95000
shipping,-7000


In [45]:
df['running_total'] = df['amount'].cumsum()
df['y_start'] = df['running_total'] - df['amount']

# Where do we want to place the label?
df['label_pos'] = df['running_total']

In [46]:
df

,amount,running_total,y_start,label_pos
sales,350000,350000,0,350000
returns,-30000,320000,350000,320000
credit fees,-7500,312500,320000,312500
rebates,-25000,287500,312500,287500
late charges,95000,382500,287500,382500
shipping,-7000,375500,382500,375500


In [47]:
df_net = pd.DataFrame.from_records([(net, net, 0, net)],
                                   columns=['amount', 'running_total', 'y_start', 'label_pos'],
                                   index=["net"])
df = df.append(df_net)

In [48]:
df

,amount,running_total,y_start,label_pos
sales,350000,350000,0,350000
returns,-30000,320000,350000,320000
credit fees,-7500,312500,320000,312500
rebates,-25000,287500,312500,287500
late charges,95000,382500,287500,382500
shipping,-7000,375500,382500,375500
net,375500,375500,0,375500


In [49]:
df['color'] = 'grey'
df.loc[df.amount < 0, 'color'] = 'red'
df.loc[df.amount < 0, 'label_pos'] = df.label_pos - 10000
df["bar_label"] = df["amount"].map('{:,.0f}'.format)

In [50]:
df

,amount,running_total,y_start,label_pos,color,bar_label
sales,350000,350000,0,350000,grey,"350,000"
returns,-30000,320000,350000,310000,red,"-30,000"
credit fees,-7500,312500,320000,302500,red,"-7,500"
rebates,-25000,287500,312500,277500,red,"-25,000"
late charges,95000,382500,287500,382500,grey,"95,000"
shipping,-7000,375500,382500,365500,red,"-7,000"
net,375500,375500,0,375500,grey,"375,500"


In [57]:
TOOLS = "box_zoom,reset,save"
source = ColumnDataSource(df)
p = figure(tools=TOOLS, x_range=list(df.index), y_range=(0, net+40000),
           plot_width=800, title = "Sales Waterfall")

In [58]:
p.segment(x0='index', y0='y_start', x1="index", y1='running_total',
          source=source, color="color", line_width=55)
show(p)

In [59]:
p.grid.grid_line_alpha=0.3
p.yaxis[0].formatter = NumeralTickFormatter(format="($ 0 a)")
p.xaxis.axis_label = "Transactions"
show(p)

In [60]:
labels = LabelSet(x='index', y='label_pos', text='bar_label',
                  text_font_size="8pt", level='glyph',
                  x_offset=-20, y_offset=0, source=source)
p.add_layout(labels)
show(p)